# Experiment 04-01

In [85]:
import sys
from glob import glob
import numpy as np
import pyopencl

## Helper Functions

In [86]:
def get_first_platform_by_name(name : str) -> pyopencl.Platform:
    """
    Get the first platform with the given name.

    Parameters
    ----------
    name : str
        The name of the platform to get.
    """
    for platform in pyopencl.get_platforms():
        if platform.name == name:
            return platform

    raise ValueError(f"No platform named '{name}'")

In [87]:
def build_program(program_files : list, context : pyopencl.Context,
        build_options=[]) -> pyopencl.Program:
    """
    Build a program from an OpenCL source file.

    Parameters
    ----------
    program_files : list
        The path to the OpenCL source files.
    context : pyopencl.Context
        The context to build the program with.
    build_options : list of str
        The build options to use.

    Returns
    -------
    pyopencl.Program
    """
    program_source_code = ''

    for ocl_file in program_files:
        with open(ocl_file, 'r') as opencl_file:
            file_source_code = opencl_file.read()
            program_source_code += '\n' + file_source_code

    program_source = pyopencl.Program(context, program_source_code)
    program = program_source.build(options=build_options)
            
    return program

In [88]:
def run_ocl_kernel(queue, kernel, global_size,
                   input_tuples, output_tuples,
                   local_size = (32,)):
    
    # copying data onto the device
    for (value, buffer) in input_tuples:
        pyopencl.enqueue_copy(queue, src=value, dest=buffer)
    
    # running program on the device
    kernel_arguments  = [buffer for (_,buffer) in input_tuples] 
    kernel_arguments += [buffer for (_,buffer) in output_tuples]
        
    kernel(queue, global_size, local_size, *kernel_arguments)

    # copying data off the device
    for (value, buffer) in output_tuples:
        pyopencl.enqueue_copy(queue, src=buffer, dest=value)
        
    # waiting for everything to finish
    queue.finish()

## Setup the device and context

In [89]:
ocl_platforms = (platform.name 
                 for platform in pyopencl.get_platforms())
print("\n".join(ocl_platforms))

Apple


In [90]:
gpu_platform = get_first_platform_by_name("Apple")
gpu_devices = gpu_platform.get_devices(device_type=pyopencl.device_type.GPU)
gpu_context = pyopencl.Context(devices=gpu_devices)

print(gpu_devices)
print(gpu_context)

[<pyopencl.Device 'Apple M1' on 'Apple' at 0x1027f00>]
<pyopencl.Context at 0x600000b08dc0 on <pyopencl.Device 'Apple M1' on 'Apple' at 0x1027f00>>


## Load Hashcat SHA256 kernel

In [91]:
build_options = [
    '-I', '.',    
]

program_files = [
    'inc_hash_sha256.cl',
    'inc_common.cl',
    'inc_platform.cl',
]

gpu_program = build_program(program_files, gpu_context, build_options)

# show the kernel names
program_kernel_names = gpu_program.get_info(pyopencl.program_info.KERNEL_NAMES)
print(f"Kernel Names: {program_kernel_names}")

Kernel Names: 
